# 1.0 Imports

In [1]:
import pandas     as pd
import numpy      as np
import seaborn    as sns
import umap.umap_ as umap

from matplotlib                import pyplot as plt
from IPython.display           import Image
from datetime                  import timedelta
from pycaret.regression        import *
from sklearn                   import cluster as c
from sklearn                   import metrics as m
from sklearn                   import ensemble as en
from sklearn                   import mixture as mx
from sklearn                   import decomposition as dd
from sklearn.manifold          import TSNE
from pandas_profiling          import ProfileReport
from sklearn.preprocessing     import MinMaxScaler, LabelEncoder, RobustScaler, StandardScaler

## 0.1 Load Data 

In [2]:
df_raw = pd.read_csv('data/marketing_campaign.csv', header=0, sep='\t') 

# 1.0 Data Description

In [3]:
df1 = df_raw.copy()

## 1.1 Data Dimensions

In [4]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of columns: {}'.format(df1.shape[1]))

Number of rows: 2240
Number of columns: 29


## 1.2 Rename Columns

In [5]:
df1.columns = df1.columns.str.lower()

## 1.3 Data Types

In [6]:
df1.dtypes

id                       int64
year_birth               int64
education               object
marital_status          object
income                 float64
kidhome                  int64
teenhome                 int64
dt_customer             object
recency                  int64
mntwines                 int64
mntfruits                int64
mntmeatproducts          int64
mntfishproducts          int64
mntsweetproducts         int64
mntgoldprods             int64
numdealspurchases        int64
numwebpurchases          int64
numcatalogpurchases      int64
numstorepurchases        int64
numwebvisitsmonth        int64
acceptedcmp3             int64
acceptedcmp4             int64
acceptedcmp5             int64
acceptedcmp1             int64
acceptedcmp2             int64
complain                 int64
z_costcontact            int64
z_revenue                int64
response                 int64
dtype: object

## 1.4 Check N/A

In [7]:
df1.isna().sum()

id                      0
year_birth              0
education               0
marital_status          0
income                 24
kidhome                 0
teenhome                0
dt_customer             0
recency                 0
mntwines                0
mntfruits               0
mntmeatproducts         0
mntfishproducts         0
mntsweetproducts        0
mntgoldprods            0
numdealspurchases       0
numwebpurchases         0
numcatalogpurchases     0
numstorepurchases       0
numwebvisitsmonth       0
acceptedcmp3            0
acceptedcmp4            0
acceptedcmp5            0
acceptedcmp1            0
acceptedcmp2            0
complain                0
z_costcontact           0
z_revenue               0
response                0
dtype: int64

## 1.5 Drop N/A

In [8]:
df1 = df1.dropna(subset=['income'])
print('Removed data: {:.2f}'.format(1-(df1.shape[0]/df_raw.shape[0])))

Removed data: 0.01


## 1.6 Change data types

In [9]:
#dt_customer
df1['dt_customer'] = pd.to_datetime(df1['dt_customer'], format='%d-%m-%Y')

## 1.7 Descriptive Statistics

In [10]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [11]:
#central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

#dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

#concat
n = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
n.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

In [12]:
n

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,id,0.0,11191.0,11191.0,5588.353339,5458.5,3248.643030,0.040459,-1.189677
1,year_birth,1893.0,1996.0,103.0,1968.820397,1970.0,11.982849,-0.353661,0.734670
2,income,1730.0,666666.0,664936.0,52247.251354,51381.5,25167.396174,6.763487,159.636700
3,kidhome,0.0,2.0,2.0,0.441787,0.0,0.536775,0.635610,-0.791164
4,teenhome,0.0,2.0,2.0,0.505415,0.0,0.544058,0.407553,-0.989619
5,recency,0.0,99.0,99.0,49.012635,49.0,28.941819,0.001648,-1.199777
6,mntwines,0.0,1493.0,1493.0,305.091606,174.5,337.251800,1.170720,0.582741
7,mntfruits,0.0,199.0,199.0,26.356047,8.0,39.784937,2.101658,4.054082
8,mntmeatproducts,0.0,1725.0,1725.0,166.995939,68.0,224.232662,2.025577,5.055477
9,mntfishproducts,0.0,259.0,259.0,37.637635,12.0,54.739727,1.916369,3.076476


In [13]:
sns.displot(data=num_attributes, x="income");

In [14]:
sns.displot(data=num_attributes, x="year_birth");

### 1.7.2 Categorical Attributes

In [15]:
sns.boxplot(x="education", y="income", data=df1, palette="Set3")

<AxesSubplot:xlabel='education', ylabel='income'>

In [16]:
sns.boxplot(x="marital_status", y="income", data=df1, palette="Set3")

<AxesSubplot:xlabel='marital_status', ylabel='income'>

# 2.0 Data Filtering

In [17]:
df2 = df1.copy()

In [18]:
#drop z_ columns
df2 = df2.drop(columns=['z_costcontact', 'z_revenue'])
#replace marital_status 'alone', 'absurd' e 'yolo' to 'single'
df2['marital_status'] = df2['marital_status'].replace(['Absurd', 'Alone', 'YOLO'], 'Single')
#delete outlier with income = 666666
df2.drop(df2.loc[df2['income'] == 666666, :].index, inplace = True)

# 3.0 Feature Engineering

In [19]:
df3 = df2.copy()

In [20]:
#monetary (total amount spend)
df3['mnttotal'] = df3['mntwines'] + df3['mntfruits'] + df3['mntmeatproducts'] + df3['mntfishproducts'] + df3['mntsweetproducts'] + df3['mntgoldprods']

#total number of purchases
df3['numtotalpurchases'] = df3['numwebpurchases'] + df3['numcatalogpurchases'] + df3['numstorepurchases'] + df3['numwebvisitsmonth']

#database generation date
df3['final_date'] = pd.to_datetime('2014-08-18')

#==========frequency============
#enrollment date - final_date = total days since the person is a client
df3['total_days'] = (df3['final_date'] - df3['dt_customer']).dt.days

#total days - recency = total period (days) that the person has been on store
df3['on_store'] = df3['total_days'] - df3['recency']

#on_store / number purchases = frequency (de quanto em quantos dias a pessoa fez uma compra)
df3['frequency'] = np.round(df3['on_store'] / df3['numtotalpurchases'])

#remove clients with 0 total purchases
df3 = df3[df3['numtotalpurchases'] != 0]

#====================date================
#day_customer
df3['day_customer'] = df3['dt_customer'].dt.day

#month_customer
df3['month_customer'] = df3['dt_customer'].dt.month

#year_customer
df3['year_customer'] = df3['dt_customer'].dt.year

#===================customers age===============
df3['customer_age'] = df3['final_date'].dt.year - df3['year_birth']

# 5.0 Data Preparation

In [21]:
df5 = df3.copy()
df5_aux = df3.copy()

## 5.1 Enconding

In [22]:
le = LabelEncoder()

#education
df5['education'] = le.fit_transform(df5['education'])

#marital_status
df5['marital_status'] = le.fit_transform(df5['marital_status'])

## 5.2 Rescaling

In [23]:
mms = MinMaxScaler()
rs = RobustScaler()
ss = StandardScaler()

#year_birth > robust scaler
df5['year_birth'] = rs.fit_transform(df5[['year_birth']].values)

#income > minmax scaler
df5['income'] = ss.fit_transform(df5[['income']].values)

#recency > minmax scaler
df5['recency'] = ss.fit_transform(df5[['recency']].values)

#mtnwines > minmax scaler 
df5['mntwines'] = ss.fit_transform(df5[['mntwines']].values)

#mntfruits > minmax scaler 
df5['mntfruits'] = ss.fit_transform(df5[['mntfruits']].values)

#mntmeatproducts > minmax scaler
df5['mntmeatproducts'] = ss.fit_transform(df5[['mntmeatproducts']].values)

#mntfishproducts > minmax scaler 
df5['mntfishproducts'] = ss.fit_transform(df5[['mntfishproducts']].values)

#mntsweetproducts > minmax scaler 
df5['mntsweetproducts'] = ss.fit_transform(df5[['mntsweetproducts']].values)

#mntgoldprods > minmax scaler 
df5['mntgoldprods'] = ss.fit_transform(df5[['mntgoldprods']].values)

#numdealspurchases > minmax scaler 
df5['numdealspurchases'] = ss.fit_transform(df5[['numdealspurchases']].values)

#numwebpurchases > minmax scaler 
df5['numwebpurchases'] = ss.fit_transform(df5[['numwebpurchases']].values)

#numcatalogpurchases > minmax scaler 
df5['numcatalogpurchases'] = ss.fit_transform(df5[['numcatalogpurchases']].values)

#numstorepurchases > minmax scaler
df5['numstorepurchases'] = ss.fit_transform(df5[['numstorepurchases']].values)

#numwebvisitsmonth > minmax scaler 
df5['numwebvisitsmonth'] = ss.fit_transform(df5[['numwebvisitsmonth']].values)

#frequency > minmax scaler 
df5['frequency'] = ss.fit_transform(df5[['frequency']].values)

#mnttotal > minmax scaler 
df5['mnttotal'] = ss.fit_transform(df5[['mnttotal']].values)

#numtotalpurchases > minmax scaler 
df5['numtotalpurchases'] = ss.fit_transform(df5[['numtotalpurchases']].values)

#customer_age > minmax scaler 
df5['customer_age'] = ss.fit_transform(df5[['customer_age']].values)

## 5.3 Transformation

In [24]:
#day
df5['day_customer_sin'] =  df5['day_customer'].apply(lambda x: np.sin(x*(2 * np.pi/30)))
df5['day_customer_cos'] =  df5['day_customer'].apply(lambda x: np.cos(x*(2 * np.pi/30)))

#month
df5['month_customer_sin'] =  df5['month_customer'].apply(lambda x: np.sin(x*(2 * np.pi/12)))
df5['month_customer_cos'] =  df5['month_customer'].apply(lambda x: np.cos(x*(2 * np.pi/12)))

#year
df5['year_customer'] = rs.fit_transform(df5[['year_customer']].values)

# 6.0 Space Study

In [25]:
df6 = df5.drop(columns=['id', 'final_date','total_days','on_store', 'dt_customer'])
X = df6.copy()

## 6.1 Tree-Based Embedding

In [26]:
#train dataset
X = df6.drop(columns=['mnttotal'], axis=1)
y = df6['mnttotal']

#model definition
rf_model = en.RandomForestRegressor(n_estimators=100, random_state=42)

#model training
rf_model.fit(X, y)

#dataframe leaf
df_leaf = pd.DataFrame(rf_model.apply(X))

In [27]:
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_leaf)

#embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

#plot UMAP
sns.scatterplot(x='embedding_x', y='embedding_y',data=df_tree)

<AxesSubplot:xlabel='marital_status', ylabel='income'>

In [28]:
df7 = df_tree.copy()
X = df7.copy()

## 9.1 Final Model

In [29]:
kmeans_list = []
#model definition
kmeans = c.KMeans(init='random', n_clusters=4, n_init=10, max_iter=300, random_state=42)

#model training
kmeans.fit(X)

#model predict
labels = kmeans.predict(X)

#performance
sil = m.silhouette_score(X, labels, metric='euclidean')
kmeans_list.append(sil)

# 11.0 Cluster Exploratory Data Analysis

In [30]:
df11 = df3.copy()
df11['cluster'] = labels

In [31]:
#number of customers
df_cluster = df11[['id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customers'] = 100*(df_cluster['id']/df_cluster['id'].sum())

#avg_income
df_avg_income = df11[['income', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_income, how='inner', on='cluster')

#avg_recency
df_avg_recency = df11[['recency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency, how='inner', on='cluster')

#avg_numdealspurchases
df_avg_numdealspurchases = df11[['numdealspurchases', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_numdealspurchases, how='inner', on='cluster')

#avg_total_spend
df_avg_spend = df11[['mnttotal', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_spend, how='inner', on='cluster')

#avg_total_purchases
df_avg_purchases = df11[['numtotalpurchases', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_purchases, how='inner', on='cluster')

#avg_age
df_avg_age = df11[['customer_age', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_age, how='inner', on='cluster')

In [32]:
df_cluster.head()

,cluster,id,perc_customers,income,recency,numdealspurchases,mnttotal,numtotalpurchases,customer_age
0,0,556,25.124266,31589.944245,48.748201,1.661871,42.541367,10.940647,42.070144
1,1,589,26.615454,75021.412564,49.923599,1.755518,1436.224109,23.293718,46.365025
2,2,532,24.039765,59033.804511,48.500000,3.362782,704.372180,22.671053,47.669173
3,3,536,24.220515,40382.070896,48.716418,2.608209,188.223881,14.434701,44.621269


# 12.0 Deploy

In [33]:
drop_columns = ['dt_customer', 'final_date', 'total_days', 'on_store','frequency','day_customer','month_customer','year_customer']
df11 = df11.drop(drop_columns, axis=1)
df12 = df11.copy()

In [34]:
import sqlite3
from sqlalchemy import create_engine

## 12.1 Insert into SQLITE 

In [39]:
df12.dtypes

id                       int64
year_birth               int64
education               object
marital_status          object
income                 float64
kidhome                  int64
teenhome                 int64
recency                  int64
mntwines                 int64
mntfruits                int64
mntmeatproducts          int64
mntfishproducts          int64
mntsweetproducts         int64
mntgoldprods             int64
numdealspurchases        int64
numwebpurchases          int64
numcatalogpurchases      int64
numstorepurchases        int64
numwebvisitsmonth        int64
acceptedcmp3             int64
acceptedcmp4             int64
acceptedcmp5             int64
acceptedcmp1             int64
acceptedcmp2             int64
complain                 int64
response                 int64
mnttotal                 int64
numtotalpurchases        int64
customer_age             int64
cluster                  int32
dtype: object

In [40]:
#create table
query_create_table_clusters = """
   CREATE TABLE clusters (
   id                  INTEGER,                       
   year_birth          INTEGER,               
   education           TEXT,               
   marital_status      TEXT,           
   income              REAL,                 
   kidhome             INTEGER,                  
   teenhome            INTEGER,                 
   recency             INTEGER,                  
   mntwines            INTEGER,                 
   mntfruits           INTEGER,                
   mntmeatproducts     INTEGER,          
   mntfishproducts     INTEGER,          
   mntsweetproducts    INTEGER,         
   mntgoldprods        INTEGER,             
   numdealspurchases   INTEGER,        
   numwebpurchases     INTEGER,          
   numcatalogpurchases INTEGER,      
   numstorepurchases   INTEGER,        
   numwebvisitsmonth   INTEGER,        
   acceptedcmp3        INTEGER,             
   acceptedcmp4        INTEGER,             
   acceptedcmp5        INTEGER,             
   acceptedcmp1        INTEGER,             
   acceptedcmp2        INTEGER,             
   complain            INTEGER,                 
   response            INTEGER,                 
   mnttotal            INTEGER,                 
   numtotalpurchases   INTEGER,
   customer_age        INTEGER,
   cluster             INTEGER)
"""

conn = sqlite3.connect('customers_clustering_db.sqlite')
conn.execute(query_create_table_clusters)
conn.commit()

#insert data
conn = create_engine('sqlite:///customers_clustering_db.sqlite')
df12.to_sql('clusters', con=conn, if_exists='append', index=False)

In [41]:
#consulting database
query = """
    SELECT * FROM clusters
"""

df = pd.read_sql_query(query, conn)